In [28]:
import numpy as np
import pandas as pd

In [2]:
# Read in the data
movies = pd.read_csv('movie.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tag.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies['movieId'].unique().size

4999

In [121]:
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,02/04/2005 23:53
1,1,29,3.5,02/04/2005 23:31
2,1,32,3.5,02/04/2005 23:33
3,1,47,3.5,02/04/2005 23:32
4,1,50,3.5,02/04/2005 23:29
...,...,...,...,...
815503,7120,168,5.0,02/04/2007 19:44
815504,7120,253,4.0,02/04/2007 19:30
815505,7120,260,5.0,02/04/2007 19:27
815506,7120,261,4.0,02/04/2007 19:49


In [9]:
movies['title'].unique().size

4999

In [15]:
ratings['userId'].unique().size

7119

In [122]:
ratings['userId'].unique().size*movies['title'].unique().size

35587881

In [38]:
su = 0
nu = 0
for i, row in ratings.iterrows():
    if(row['userId'] == 1):
        su+=row['rating']
        nu+=1
su, nu, su/nu

,userId,rating
movieId,,
1,3528.748151,3.959323
2,3520.225974,3.268398
3,3580.858394,3.186861
4,3765.543478,3.000000
5,3418.366819,3.143836


In [48]:
user_item_matrix = ratings.pivot(index='movieId', columns='userId', values='rating')

In [59]:
user_item_matrix

userId,1,2,3,4,5,6,7,8,9,10,...,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,4.0,NaN,NaN,5.0,NaN,4.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,4.5
2,3.5,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
3,NaN,4.0,NaN,NaN,NaN,3.0,3.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The movies which did not have any ratings have been removed

In [50]:
um_mat_np = user_item_matrix.to_numpy()
masks = np.isnan(um_mat_np)
masked_arr = np.ma.masked_array(um_mat_np, masks)


In [52]:
rating_means = np.mean(masked_arr, axis=1)

In [85]:
filled_matrix = (masked_arr.T).filled(rating_means).T
print(filled_matrix)
filled_matrix = filled_matrix - rating_means.data[:,np.newaxis]

[[3.95932269 3.95932269 4.         ... 3.95932269 5.         4.5       ]
 [3.5        3.26839827 3.26839827 ... 3.26839827 3.26839827 4.        ]
 [3.18686131 4.         3.18686131 ... 3.18686131 3.18686131 3.18686131]
 ...
 [2.77272727 2.77272727 2.77272727 ... 2.77272727 2.77272727 2.77272727]
 [2.55633803 2.55633803 2.55633803 ... 2.55633803 2.55633803 2.55633803]
 [2.         2.         2.         ... 2.         2.         2.        ]]


In [86]:
rating_means

masked_array(data=[3.959322693655119, 3.2683982683982684,
                   3.1868613138686133, ..., 2.772727272727273,
                   2.556338028169014, 2.0],
             mask=[False, False, False, ..., False, False, False],
       fill_value=1e+20)

In [87]:
filled_matrix

array([[0.        , 0.        , 0.04067731, ..., 0.        , 1.04067731,
        0.54067731],
       [0.23160173, 0.        , 0.        , ..., 0.        , 0.        ,
        0.73160173],
       [0.        , 0.81313869, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [107]:
u, s, vh = np.linalg.svd(filled_matrix, full_matrices=False)

In [109]:
u[0]@ (np.diag(s)@vh)

array([-3.68511580e-15, -3.93914806e-15,  4.06773063e-02, ...,
        4.44454963e-16,  1.04067731e+00,  5.40677306e-01])

In [113]:
v = vh.T

In [112]:
s_sqrt = s**0.5
s_sqrt_mt = np.diag(s_sqrt)

In [119]:
k = 1000
U = u[:,:k]@s_sqrt_mt[:k,:k]

V = v[:,:k]@s_sqrt_mt[:k,:k]

In [120]:
U[0]@V.T

array([-0.01538027,  0.01618134,  0.04114364, ...,  0.0074478 ,
        1.0226725 ,  0.53218587])